# CARP

In [1]:
%cd ~/repo/protein-transfer

/home/t-fli/repo/protein-transfer


In [2]:
%load_ext blackcellmagic

In [18]:
!pip install sequence-models

In [3]:
from scr.encoding.encoding_classes import CARPEncoder

In [4]:
from scr.utils import pickle_load

In [5]:
df = pickle_load("data/proeng/gb1/two_vs_rest.pkl")

In [6]:
df_train = df.loc[(df["set"] == "train") & (df["validation"] != True)]
df_val = df.loc[(df["set"] == "train") & (df["validation"] == True)]
df_test = df.loc[(df["set"] == "test")]

len(df_train), len(df_val), len(df_test), len(df)

(381, 43, 8309, 8733)

In [7]:

seqs = [[seq] for seq in df_val.sequence.astype(str).str[0 : 56].values[0:2]]
seqs

[['MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVNGEWTYDDATKTFTVTE'],
 ['MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGEYGEWTYDDATKTFTVTE']]

In [8]:
list(df_val.sequence.astype(str).str[0 : 56].values[0:2])

['MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVNGEWTYDDATKTFTVTE',
 'MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGEYGEWTYDDATKTFTVTE']

In [12]:
no_flat_encoder = CARPEncoder(
    encoder_name="carp_600k",
).encode(mut_seqs=list(df_val.sequence.astype(str).str[0 : 56].values[0:2]))
one_emb = next(no_flat_encoder)
one_emb[0], one_emb[0].shape

Generating carp_600k upto 16 layer embedding ...
No embedding flattening
No embedding flattening
No embedding flattening
No embedding flattening
No embedding flattening
No embedding flattening
No embedding flattening
No embedding flattening
No embedding flattening
No embedding flattening
No embedding flattening
No embedding flattening
No embedding flattening
No embedding flattening
No embedding flattening
No embedding flattening


(array([[[ 0.11233799, -0.0309646 ,  0.09588872, ..., -0.21515834,
          -0.41078556,  0.841719  ],
         [ 0.2050094 , -0.14373398, -0.15035415, ...,  0.13152511,
          -0.129341  ,  0.5012831 ],
         [ 0.4236133 , -0.09426638,  0.04951385, ...,  0.22084066,
          -3.9057858 ,  0.7927067 ],
         ...,
         [ 0.31486928, -0.00696394,  0.12247564, ..., -0.05261476,
          -4.633561  ,  0.5593643 ],
         [ 0.27489308, -0.01615848,  0.14888932, ..., -0.03519903,
           0.3777457 ,  0.5697537 ],
         [ 0.34990993,  0.25660816,  0.21758933, ...,  0.10277103,
          -3.0541046 ,  0.3111604 ]],
 
        [[ 0.11233799, -0.0309646 ,  0.09588872, ..., -0.21515834,
          -0.41078556,  0.841719  ],
         [ 0.2050094 , -0.14373398, -0.15035415, ...,  0.13152511,
          -0.129341  ,  0.5012831 ],
         [ 0.4236133 , -0.09426638,  0.04951385, ...,  0.22084066,
          -3.9057858 ,  0.7927067 ],
         ...,
         [ 0.31486928, -0.0069639

In [15]:
mean_flat_encoder = CARPEncoder(
    encoder_name="carp_600k",
).encode(mut_seqs=list(df_val.sequence.astype(str).str[0 : 56].values[0:2]),flatten_emb="mean")
one_mean_emb = next(mean_flat_encoder)
one_mean_emb[0].shape

Generating carp_600k upto 16 layer embedding ...


(2, 128)

In [10]:
from sequence_models.pretrained import load_model_and_alphabet

model, collater = load_model_and_alphabet('carp_600k')

x = collater(seqs)[0]  # (n, max_len)
# rep = model(x)  # (n, max_len, d_model)

activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

model.model.embedder.layers[0].register_forward_hook(get_activation("layer0"))

rep = model(x)

activation["layer0"], activation["layer0"].shape

(tensor([[[ 0.1123, -0.0310,  0.0959,  ..., -0.2152, -0.4108,  0.8417],
          [ 0.2050, -0.1437, -0.1504,  ...,  0.1315, -0.1293,  0.5013],
          [ 0.4236, -0.0943,  0.0495,  ...,  0.2208, -3.9058,  0.7927],
          ...,
          [ 0.3149, -0.0070,  0.1225,  ..., -0.0526, -4.6336,  0.5594],
          [ 0.2749, -0.0162,  0.1489,  ..., -0.0352,  0.3777,  0.5698],
          [ 0.3499,  0.2566,  0.2176,  ...,  0.1028, -3.0541,  0.3112]],
 
         [[ 0.1123, -0.0310,  0.0959,  ..., -0.2152, -0.4108,  0.8417],
          [ 0.2050, -0.1437, -0.1504,  ...,  0.1315, -0.1293,  0.5013],
          [ 0.4236, -0.0943,  0.0495,  ...,  0.2208, -3.9058,  0.7927],
          ...,
          [ 0.3149, -0.0070,  0.1225,  ..., -0.0526, -4.6336,  0.5594],
          [ 0.2749, -0.0162,  0.1489,  ..., -0.0352,  0.3777,  0.5698],
          [ 0.3499,  0.2566,  0.2176,  ...,  0.1028, -3.0541,  0.3112]]]),
 torch.Size([2, 56, 128]))

In [19]:
model(x, repr_layers=[0])[0]

TypeError: forward() got an unexpected keyword argument 'repr_layers'